# 영상의 특징 추출
### edge 
- 영상에서 픽셀의 밝기 값이 급격하게 변하는 부분
- 일반적으로 배경과 객체, 또는 객체와 객체의 경계
### 미분을 이용한 edge 검출
- 영상을 (x,y) 변수의 함수로 간주했을 때, 이 함수의 1차 미분값이 크게 나타나는 부분을 검출

### Sobel filter

In [1]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
src = cv2.imread("../data/lenna.bmp", cv2.IMREAD_GRAYSCALE)

dx = cv2.Sobel(src, -1, 1, 0, delta=128)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)

cv2.imshow("src", src)
cv2.imshow("dx", dx)
cv2.imshow("dy", dy)

cv2.waitKey()
cv2.destroyAllWindows()

### 그레디언트
- 함수 f(x,y)를 x축과 y축으로 각각 편미분 하여 벡터 형태로 표현한 것.
- 영상에서 그레디언트?
- 그레디언트 크기 : 픽셀 값의 차이 정도, 변화량
- 그레디언트 방향 : 픽셀 값이 가장 급격하게 증가하는 방향

### Sobel edge extraction

In [6]:
src = cv2.imread("../data/lenna.bmp", cv2.IMREAD_GRAYSCALE)

dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0, 255).astype(np.uint8)
_, dst = cv2.threshold(mag, 120, 255, cv2.THRESH_BINARY)

cv2.imshow("src", src)
cv2.imshow("mag", mag)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### canny edge extraction
- 1단계
    - 가우시안 필터링
        - (optional) 잡음 제거 목적
- 2단계
    - 그레디언트 계산
        - 주로 sobel mask 사용
- 3단계
    - 비최대 억제 (non-maximum suppression)
        - 하나의 에지가 여러 개의 픽셀로 표현되는 현상을 없애기 위해 그래디언트 크기가 국지적 최대인 픽셀만 에지 픽셀로 설정
        - 그래디언트 방향에 위치한 두 개의 픽셀을 조사하여 국지적 최대를 검사
- 4단계
    - 히스테리시스 에지 트래킹 
        - 두 개의 임계값을 사용

In [8]:
src = cv2.imread("../data/building.jpg", cv2.IMREAD_GRAYSCALE)
dst = cv2.Canny(src, 50, 150)

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### 허프 변환 직선 검출
- 2차원 영상 좌표에서 직선의 방정식을 파라미터 공간으로 변환하여 직선을 찾는 알고리즘

In [12]:
src = cv2.imread("../data/building.jpg", cv2.IMREAD_GRAYSCALE)

edges = cv2.Canny(src, 50, 150)

lines = cv2.HoughLinesP(edges, 1, np.pi / 180., 160, 
                        minLineLength=50, maxLineGap=5)

dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1]) # 시작점 좌표
        pt2 = (lines[i][0][2], lines[i][0][3]) # 끝점 좌표
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### 허프 원 검출

In [4]:
src = cv2.imread("../data/dial.jpg")

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)

def on_trackbar(pos):
    rmin = cv2.getTrackbarPos("minRadius", "img")
    rmax = cv2.getTrackbarPos("maxRadius", "img")
    th = max(1, cv2.getTrackbarPos("threshold", "img"))  # 0이 되지 않게


    circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50, 
                               param1=120, param2=th, minRadius=rmin, maxRadius=rmax)
    
    dst = src.copy()
    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = map(int, circles[0][i])  # 정수형으로 변환
            cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)
            
    cv2.imshow("img", dst)

# 트랙바 생성
cv2.imshow('img', src)
cv2.createTrackbar('minRadius', 'img', 0, 100, on_trackbar)
cv2.createTrackbar('maxRadius', 'img', 0, 150, on_trackbar)
cv2.createTrackbar('threshold', 'img', 0, 100, on_trackbar)
cv2.setTrackbarPos('minRadius', 'img', 10)
cv2.setTrackbarPos('maxRadius', 'img', 80)
cv2.setTrackbarPos('threshold', 'img', 40)

cv2.waitKey()
cv2.destroyAllWindows()

### 신기한 예제. 동전 카운터

In [ ]:
import cv2
import numpy as np

src = cv2.imread("../data/coins2.jpeg")

# 이미지 확대
scale = 4.0
src = cv2.resize(src, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)

def on_trackbar(pos):
    rmin = cv2.getTrackbarPos("Radius min", "dst")
    rmax = cv2.getTrackbarPos("Radius max", "dst")
    th = max(1, cv2.getTrackbarPos("threshold", "dst"))
    hue1 = cv2.getTrackbarPos("Hue < 1", "dst")
    hue2 = cv2.getTrackbarPos("Hue < 2", "dst")
    hue3 = cv2.getTrackbarPos("Hue < 3", "dst")

    circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,
                               param1=150, param2=th,
                               minRadius=rmin, maxRadius=rmax)

    dst = src.copy()
    sum_of_money = 0

    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = map(int, circles[0][i])
            cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)

            x1 = max(cx - radius, 0)
            y1 = max(cy - radius, 0)
            x2 = min(cx + radius, dst.shape[1])
            y2 = min(cy + radius, dst.shape[0])
            radius = int(radius)

            crop = dst[y1:y2, x1:x2, :]
            ch, cw = crop.shape[:2]

            mask = np.zeros((ch, cw), np.uint8)
            cv2.circle(mask, (cw//2, ch//2), radius, 255, -1)

            hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
            hue, _, _ = cv2.split(hsv)
            mean_of_hue = cv2.mean(hue, mask)[0]

            # 수정된 Hue 기준
            if mean_of_hue < hue1:
                won = 50
            elif mean_of_hue < hue2:
                won = 10
            elif mean_of_hue < hue3:
                won = 500
            else:
                won = 100

            sum_of_money += won

            cv2.putText(dst, f"{won}won", (cx - 30, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.putText(dst, f"sum_of_money: {sum_of_money}won", (30, 60),
                cv2.FONT_HERSHEY_DUPLEX, 1.5, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("dst", dst)

# 트랙바 생성
cv2.imshow("dst", src)
cv2.createTrackbar("Radius min", "dst", 0, 200, on_trackbar)
cv2.createTrackbar("Radius max", "dst", 0, 200, on_trackbar)
cv2.createTrackbar("threshold", "dst", 1, 100, on_trackbar)
cv2.createTrackbar("Hue < 1", "dst", 0, 179, on_trackbar)
cv2.createTrackbar("Hue < 2", "dst", 0, 179, on_trackbar)
cv2.createTrackbar("Hue < 3", "dst", 0, 179, on_trackbar)

# 초기값 설정
cv2.setTrackbarPos("Radius min", "dst", 95)
cv2.setTrackbarPos("Radius max", "dst", 146)
cv2.setTrackbarPos("threshold", "dst", 49)
cv2.setTrackbarPos("Hue < 1", "dst", 14)
cv2.setTrackbarPos("Hue < 2", "dst", 48)
cv2.setTrackbarPos("Hue < 3", "dst", 145)

on_trackbar(0)
cv2.waitKey()
cv2.destroyAllWindows()